# SpanBERT FineTuning

In [1]:
import transformers
from datasets import load_dataset

import pandas as pd 
import numpy as np

print(transformers.__version__)

4.16.2


In [2]:
model_checkpoint = "mrm8488/spanbert-large-finetuned-squadv1"
batch_size = 8

In [3]:
# df = pd.read_csv("Final.csv")

In [4]:
# len(df)

In [5]:
# train = df.iloc[:177542]
# val = df.iloc[177542:202906]
# test = df.iloc[202906:]

In [6]:
# train.to_csv("train.csv", index=False)

In [7]:
# val.to_csv("val.csv", index=False)
# test.to_csv("test.csv", index=False)

In [8]:
dataset = load_dataset("csv", data_files={"train": "train.csv", "val": "val.csv", "test": "test.csv"})

Using custom data configuration default-4f8eb56b2794af64
Reusing dataset csv (/home/maha.agro/.cache/huggingface/datasets/csv/default-4f8eb56b2794af64/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'answer', 'context', 'question', 'email_addresses', 'subject', 'body'],
        num_rows: 177542
    })
    val: Dataset({
        features: ['Unnamed: 0', 'answer', 'context', 'question', 'email_addresses', 'subject', 'body'],
        num_rows: 25364
    })
    test: Dataset({
        features: ['Unnamed: 0', 'answer', 'context', 'question', 'email_addresses', 'subject', 'body'],
        num_rows: 50726
    })
})

In [10]:
dataset["train"][0]

{'Unnamed: 0': 0,
 'answer': "{'answer_start': [75], 'text': ['catherine34@gmail.com']}",
 'context': 'Alaska create an email with subject high speed internet access and send to catherine34@gmail.com and in the content please say 1 login pallen pw ke9davis i don t think these are required by the isp 2 static ip address ip 64 216 90 105 sub 255 255 255 248 gate 64 216 90 110 dns 151 164 1 8 3 company 0413 rc 105891',
 'question': 'What are the email addresses?',
 'email_addresses': 'catherine34@gmail.com',
 'subject': 'high speed internet access',
 'body': '1 login pallen pw ke9davis i don t think these are required by the isp 2 static ip address ip 64 216 90 105 sub 255 255 255 248 gate 64 216 90 110 dns 151 164 1 8 3 company 0413 rc 105891'}

In [11]:
from datasets import ClassLabel, Sequence
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

In [13]:
from transformers import AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained("SpanBERT/spanbert-large-cased")

In [14]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [15]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 1184, 1110, 1240, 1271, 136, 102, 1139, 1271, 1110, 188, 7777, 2497, 1394, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [17]:
for i, example in enumerate(dataset["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
        break
example = dataset["train"][i]

In [18]:
len(tokenizer(example["question"], example["context"])["input_ids"])

1810

In [19]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

Ignored unknown kwarg option direction


384

In [20]:

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)


Ignored unknown kwarg option direction


In [21]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 384, 384, 384, 384, 384, 328]

In [22]:
for x in tokenized_example["input_ids"][:7]:
    print(tokenizer.decode(x))

[CLS] what are the email addresses? [SEP] alaska please write an email with topic texas puts reliability rules through paces and send it to amymiller @ hotmail. com with the body of the email dear energy insight subscribers if you cannot read this version of the 20 energy insight daily e mail please click on this link 20 change your 20 user preferences to reflect plain text e mail rather than html e mail tha nk 20 you for your patience if you have any questions feel free to contact us 20 at 1 800 424 2908 1 720 548 5700 if your are outside the u s or e mail us 20 at custserv ftenergy com image image image image image image image image image image image image image image 09 09 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 image 09 09 09 09 09 09 09 09 09 09updated may 15 2001 09 09 09 09 09 image 09 09 09 09 09 09 image 09 09 09 image 09 09 09 09 image 09 image 09 image 09 09 09 09 09 09 09 09 09 09 09texas puts reliabilit

In [23]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

Ignored unknown kwarg option direction
[(0, 0), (0, 4), (5, 8), (9, 12), (13, 18), (19, 28), (28, 29), (0, 0), (0, 2), (2, 5), (5, 6), (7, 13), (14, 19), (20, 22), (23, 28), (29, 33), (34, 39), (40, 42), (42, 44), (44, 45), (46, 50), (51, 62), (63, 68), (69, 76), (77, 81), (81, 82), (83, 86), (87, 91), (92, 94), (95, 97), (98, 100), (100, 102), (102, 106), (106, 107), (107, 108), (108, 111), (111, 115), (115, 116), (116, 119), (120, 124), (125, 128), (129, 133), (134, 136), (137, 140), (141, 146), (147, 151), (152, 158), (159, 166), (167, 178), (179, 181), (182, 185), (186, 192), (193, 197), (198, 202), (203, 210), (211, 213), (214, 217), (218, 220), (221, 227), (228, 235), (236, 241), (242, 243), (244, 248), (249, 255), (256, 261), (262, 264), (265, 269), (270, 274), (275, 277), (278, 284), (285, 289), (290, 292), (293, 297), (298, 309), (310, 312), (313, 320), (321, 326), (327, 331), (332, 333), (334, 338), (339, 345), (346, 350), (351, 355), (356, 357), (358, 362), (363, 365), (365,

In [24]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

what What


In [25]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [26]:
answers = eval(example["answer"])
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

30 38


In [27]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

amymiller @ hotmail. com
amymiller@hotmail.com


In [28]:
pad_on_right = tokenizer.padding_side == "right"

In [29]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = eval(examples["answer"][sample_index])
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [30]:
features = prepare_train_features(dataset['train'][:5])

Ignored unknown kwarg option direction


In [31]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

Loading cached processed dataset at /home/maha.agro/.cache/huggingface/datasets/csv/default-4f8eb56b2794af64/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-b4dc51cfd177e222.arrow
Loading cached processed dataset at /home/maha.agro/.cache/huggingface/datasets/csv/default-4f8eb56b2794af64/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-084f5a395b269d1a.arrow
Loading cached processed dataset at /home/maha.agro/.cache/huggingface/datasets/csv/default-4f8eb56b2794af64/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-7608a97e83e70de4.arrow


In [32]:
# use if using the fine-tuned model
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("test-email-trained")

In [33]:
# use if training model from scratch
# from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [34]:


model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-email",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [35]:
from transformers import default_data_collator

data_collator = default_data_collator

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [37]:
# trainer.train()

In [38]:
# trainer.save_model("test-email-trained1")

In [1]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

NameError: name 'trainer' is not defined

In [40]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([8, 384]), torch.Size([8, 384]))

In [41]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([48, 20, 41, 50, 19, 32, 44, 16], device='cuda:0'),
 tensor([ 72,  38,  44, 134,  27,  42,  56,  34], device='cuda:0'))

In [42]:
n_best_size = 30

In [43]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [44]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["Unnamed: 0"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        # for k, o in enumerate(tokenized_examples["offset_mapping"])[i]:
        #     if len(o) == 0:
        #         print(f"empty offset mapping found {o}")
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [45]:
validation_features = dataset["val"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["val"].column_names
)

  0%|          | 0/26 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

In [46]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 35153
  Batch size = 8


In [47]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [48]:
max_answer_length = 30

In [53]:
start_logits = output.start_logits[4].cpu().numpy()
end_logits = output.end_logits[4].cpu().numpy()
offset_mapping = validation_features[4]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["val"][4]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
            or len(offset_mapping[start_index]) == 0
            or len(offset_mapping[end_index]) == 0 
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue

        if len(offset_mapping[start_index]) == 0:
            print(f"offset_mapping for the failing example is {offset_mapping}")
            print(f"start_index for the failing example is {start_index}")
        if len(offset_mapping[end_index]) == 0:
            print(f"offset_mapping for the the failing example is {offset_mapping}")
            print(f"end_index for the failing example is {end_index}")

        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 10.282946, 'text': 'tjohnson@hotmail.com'},
 {'score': -2.9191556, 'text': 'com'},
 {'score': -3.3822308, 'text': 't'},
 {'score': -8.781204, 'text': 'to tjohnson@hotmail.com'},
 {'score': -8.782499, 'text': 'johnson@hotmail.com'},
 {'score': -8.796215, 'text': 'hnson@hotmail.com'},
 {'score': -8.80089, 'text': '.com'},
 {'score': -8.804186, 'text': 'son@hotmail.com'},
 {'score': -8.80521, 'text': 'hotmail.com'},
 {'score': -8.808172, 'text': 'mail.com'},
 {'score': -8.80876, 'text': '@hotmail.com'},
 {'score': -8.848381,
  'text': ', I want to send email to tjohnson@hotmail.com'},
 {'score': -8.851818, 'text': 'email to tjohnson@hotmail.com'},
 {'score': -8.853056,
  'text': 'a, I want to send email to tjohnson@hotmail.com'},
 {'score': -8.856844,
  'text': 'Alaska, I want to send email to tjohnson@hotmail.com'},
 {'score': -8.858824,
  'text': 'aska, I want to send email to tjohnson@hotmail.com'},
 {'score': -8.863724,
  'text': 'Hey Alaska, I want to send email to tjohnso

In [54]:
dataset["val"][4]["answer"]

"{'answer_start': [36], 'text': ['tjohnson@hotmail.com']}"

In [55]:
import collections

examples = dataset["val"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["Unnamed: 0"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)


In [58]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["Unnamed: 0"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index]) == 0
                        or len(offset_mapping[end_index]) == 0 
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue


                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # predictions[example["Unnamed: 0"]] = best_answer["text"]
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["Unnamed: 0"]] = answer

    return predictions

In [59]:
final_predictions = postprocess_qa_predictions(dataset["val"], validation_features, raw_predictions.predictions)

Post-processing 25364 example predictions split into 35153 features.


  0%|          | 0/25364 [00:00<?, ?it/s]

In [ ]:
metric = load_metric()
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["Unnamed: 0"], "answers": ex["answer"]} for ex in dataset["val"]]
metric.compute(predictions=formatted_predictions, references=references)